# Notebook for Creating High Wage Outputs

#### This notebook is currently written to create high wage outputs for 2020.

In [1]:
import pandas as pd
import numpy as np
import string
import warnings
import os
import re
from jqi_functions import *
warnings.filterwarnings('ignore')

## Ensure that IPUMS data is in the proper file location

The desired year for IPUMS data should live in the data folder, under `data/ipums` with the naming convention as `IPUMS_{YEAR}.csv`, where `{YEAR}` should match the year entered below.

A full example for the file path would be `data/ipums/IPUMS_2020.csv`.

## Set the desired year and the corresponding cost of living year

In [2]:
year = '2021'

## Creating IPUMS dataframe

#### IPUMS Data
`cleaned_ipums` is a function to generate a cleaned pandas dataframe using IPUMS data, filtering it down to California only and the desired year. The year needs to be entered in string format as a parameter.

In [3]:
ca_ipums = cleaned_ipums(year)

KeyError: 2021

#### Cost of living needs to be updated each year.

In [ ]:
cost_of_living = pd.read_csv(f'data/cost_of_living/united-way-col-1A1PS1C{year}.csv')

### Create county lookup dataframe

Expanding the `county_info` dataframe to include cost of living metrics. This dataframe is used when industry information in a geographic area is too sparse and the next largest geographic area needs to be used instead.

In [ ]:
county_info = pd.read_csv('data/county_to_regions_key.csv')

In [ ]:
county_info = county_info[['County', 'COUNTYFIP', 'Rural/Urban', 'CERF Regions', 'Population']]

In [ ]:
county_info = pd.merge(county_info, cost_of_living, left_on = 'CERF Regions', right_on = 'Regions')

In [ ]:
county_info = county_info.rename(columns = {'Cost of Living':'Regional COL'})
county_info = county_info.drop(columns=['Regions'])

In [ ]:
county_info['State COL'] = cost_of_living.iloc[13][1]

View of final `county_info` dataframe.

In [ ]:
county_info.head()

In [ ]:
ca_ipums = pd.merge(ca_ipums, county_info, on = 'COUNTYFIP')

View of final `ca_ipums` dataframe.

In [ ]:
ca_ipums.head()

## Create EDD Dataframe

#### EDD Data
The year for EDD data must be specified.

These CSV files are filtered and cleaned versions of the raw EDD Current Employment Statistics dataset. These CSV files can be created for upcoming years with the notebook `multiyear-edd-data-creation.ipynb`

In [ ]:
edd = pd.read_csv(f'data/edd/edd_{year}.csv')

View of final `edd` dataframe.

In [ ]:
edd.head()

## Add High Wage Features

`add_geo_high_wages` is a function that adds the following engineered features:
- Above Threshold (Number of records above respective cost of living threshold)
- Weighted above threshold (Above Threshold multiplied by person weight variable)
- Unweighted industry counts (Number of records in that industry)
- Weighted industry counts (Sum of person weight values in that industry)
- Weighted high wage percentage (Weighted Above Threshold divided by Weighted Industry Counts as a percentage)

The features are created for the following geographical levels:
- Region
- California

In [ ]:
ca_ipums_hw = add_geo_high_wages(ca_ipums)

View of final `ca_ipums_hw` dataframe.

In [ ]:
ca_ipums_hw.head().T

## Create High Wage Outputs Dataframe

`edd_to_hw` is the function that outputs the values needed to create the high wage output dataframe. This portion of the notebook runs through every unique combination of region, industry, and date, to get that respective output and add it to the dataframe.

Because of the nested structure of the EDD industries, only a small selection of EDD industries can be used to ensure that individuals in nested industries are not counted twice. The selection of these industries different per region, so the series code of each industry is documented in the `region_series_codes` global variable in the `jqi_functions.py` library. Each of these codes were then assigned their own crosswalk value, which align with each crosswalk value assigned to each IPUMS industry. For generating high wage outputs, we only iterate through the EDD industries that have been selected and have a designated crosswalk value.

Getting unique values for each region, industry, and date.

In [ ]:
regions_ipums = ca_ipums_hw['CERF Regions'].unique()

In [ ]:
crosswalk_vals = sorted(edd['Crosswalk Value'].unique())

In [ ]:
dates_edd = edd['Date'].unique()

Initializing empty lists for the function's outputs to later be joined in a dataframe.

In [ ]:
industries = []
dates = []
regions = []
counts = []
emp_counts = []

In [ ]:
total_iterations = len(regions_ipums) * len(crosswalk_vals) * len(dates_edd)

For loop to populate lists for the high wage outputs. This will take some time to finish running.

In [ ]:
progress_count = 0
for region in regions_ipums:
    for code in crosswalk_vals:
        for date in dates_edd:
            hw_count, hw_perc, employment_count, industry = edd_to_hw(edd, ca_ipums_hw, region, code, date, 10)
            industries.append(industry)
            dates.append(date)
            regions.append(region)
            counts.append(hw_count)
            emp_counts.append(employment_count)
            progress_count += 1
            if progress_count % (total_iterations / 10) == 0:
                percent_done = int((progress_count / total_iterations) * 100)
                print(f'Progress: {percent_done}% Complete')

Creating a cleaned dataframe from the output lists.

In [ ]:
df_dict = {'Industry':industries, 'Date':dates, 'Region':regions, 'High Wage Count':counts, 'Employment Count':emp_counts}
hw_output = pd.DataFrame(df_dict)
hw_output = hw_output[hw_output['Industry'].notna()]
hw_output['Date']= pd.to_datetime(hw_output['Date'])
hw_output['High Wage Count'] = hw_output['High Wage Count'].astype(int)
hw_output = hw_output.sort_values(by=['Industry', 'Region', 'Date'])
hw_output = pd.merge(hw_output, cost_of_living, left_on='Region', right_on='Regions')
hw_output = hw_output[['Industry', 'Date', 'Region', 'High Wage Count', 'Employment Count', 'Cost of Living']]
hw_output = hw_output.drop_duplicates()

Add Region Population to Dataframe

In [ ]:
reg_pop = county_info.groupby(by='CERF Regions').sum()[['Population']].reset_index()
reg_pop

In [ ]:
hw_output = pd.merge(hw_output, reg_pop, left_on='Region', right_on='CERF Regions')
hw_output = hw_output.drop(columns=['CERF Regions'])
hw_output = hw_output.rename(columns={"Population": "Region Population"})

View of final `hw_output` dataframe.

In [ ]:
hw_output.head()

Code to export the dataframe as a CSV file - change file path if needed and uncomment to run.

In [ ]:
hw_output.to_csv(f'data/outputs/hw_outputs_{year}.csv', encoding='utf-8', index=False)

#### TESTING

In [ ]:
bayarea = hw_output.loc[hw_output['Date'] == '2021-12-01'].loc[hw_output['Region'] == 'Bay Area']

In [ ]:
bayarea['High Wage Count'].sum()

In [ ]:
bayarea['Employment Count'].sum()

In [ ]:
california = hw_output.loc[hw_output['Date'] == '2021-12-01']

In [ ]:
california['High Wage Count'].sum()

In [ ]:
california['Employment Count'].sum()

https://data.bls.gov/timeseries/LASST060000000000005?amp%253bdata_tool=XGtable&output_view=data&include_graphs=true

## Code for concatenating multiple years

In [ ]:
# hw_2010 = pd.read_csv('data/outputs/hw_outputs_2010.csv')
# hw_2011 = pd.read_csv('data/outputs/hw_outputs_2011.csv')
# hw_2012 = pd.read_csv('data/outputs/hw_outputs_2012.csv')
# hw_2013 = pd.read_csv('data/outputs/hw_outputs_2013.csv')
# hw_2014 = pd.read_csv('data/outputs/hw_outputs_2014.csv')
# hw_2015 = pd.read_csv('data/outputs/hw_outputs_2015.csv')
# hw_2016 = pd.read_csv('data/outputs/hw_outputs_2016.csv')
# hw_2017 = pd.read_csv('data/outputs/hw_outputs_2017.csv')
# hw_2018 = pd.read_csv('data/outputs/hw_outputs_2018.csv')
# hw_2019 = pd.read_csv('data/outputs/hw_outputs_2019.csv')
# hw_2020 = pd.read_csv('data/outputs/hw_outputs_2020.csv')
# hw_2021 = pd.read_csv('data/outputs/hw_outputs_2021.csv')

In [ ]:
# hw_2010['Year'] = 2010
# hw_2011['Year'] = 2011
# hw_2012['Year'] = 2012
# hw_2013['Year'] = 2013
# hw_2014['Year'] = 2014
# hw_2015['Year'] = 2015
# hw_2016['Year'] = 2016
# hw_2017['Year'] = 2017
# hw_2018['Year'] = 2018
# hw_2019['Year'] = 2019
# hw_2020['Year'] = 2020
# hw_2021['Year'] = 2021

In [ ]:
# hw_output_concat = pd.concat([hw_2010, hw_2011, hw_2012, hw_2013, 
#                               hw_2014, hw_2015, hw_2016, hw_2017, 
#                               hw_2018, hw_2019, hw_2020, hw_2021])

In [ ]:
# hw_output_concat.to_csv('data/outputs/hw_outputs_multiyear.csv', encoding='utf-8', index=False)